# Atividade 2 - Visão Computacional

In [1]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np
import time as t
import sys
import math
print ("OpenCV Version : %s " % cv2.__version__)

from ipywidgets import widgets, interact, interactive, FloatSlider, IntSlider

import auxiliar as aux

if (sys.version_info > (3, 0)): 
    # Modo Python 3
    import importlib
    importlib.reload(aux) # Para garantir que o Jupyter sempre relê seu trabalho
else:
    # Modo Python 2
    reload(aux)

OpenCV Version : 3.4.2 


O entregável de toda esta atividade vai ser um código-fonte em C. 

Encorajamos vocês a fazerem vídeos demonstrando o resultado e a postar (pode ser privadamente) no YouTube



In [2]:
! explorer .


Você deve ter uma folha com o padrão anexo. 
*Dica:* Se não tiver, é possível fazer também com um tablet ou *smartphone*
 
<img src="folha_atividade.png" width=300>

## Parte 1 - calibração

Ouça a explicação do professor sobre o modelo de câmera *pinhole*  e desenhe a medida $f$ que separa o plano focal da pupila da câmera

Detalhe como calculou $f$

![](circulos.png)

In [3]:
#Medidas
ho = 14
d = 21

In [4]:
#Coordenadas círculo 1
x1 = 493
y1 = 287

#Coordenadas círculo 2
x2 = 163
y2 = 293

In [5]:
#Normalizando a coordenada y
y = y2-y1
y

6

In [6]:
#Normalizando a coordenada x
x = x1-x2
x

330

In [7]:
#Calculando a distância entre os círculos por Pitágoras
hi = math.sqrt(y**2 + x**2)
hi

330.0545409474016

In [8]:
#Calculando f
f = hi*d/ho
f

495.08181142110243

Para testar o resultado obtido, vamos fazer o teste para calcular d, sabendo que o d medido foi de 18cm.

![](circulos_teste.png)

In [9]:
#Coordenadas círculo 1
tx1 = 113
ty1 = 273

#Coordenadas círculo 2
tx2 = 499
ty2 = 245

In [10]:
#Normalizando a coordenada y
ty = ty1-ty2
ty

28

In [11]:
#Normalizando a coordenada x
tx = tx2-tx1
tx

386

In [12]:
#Calculando a distância entre os círculos por Pitágoras
hi_ = math.sqrt(ty**2 + tx**2)
hi_

387.0142116253614

In [13]:
#Para calcular d
d = f*ho/hi_
d

17.90927865616713

O valor encontrado chegou bem próximo do valor medido, e, portanto, podemos dizer que o algoritmo funciona.

## Parte 2

Modifique um dos exemplos `draw_circles_video.py` ou `videoplay.py` para passar a ler dados da webcam e identificar o círculo magenta e o círculo ciano, usando o `inRange`

In [14]:
#Dados obtidos pela função colorpicker para o círculo magenta
hsv1_m = np.array([150,50,50], dtype=np.uint8)
hsv2_m = np.array([220,255,255], dtype=np.uint8)

In [15]:
#Dados obtidos pela função colorpicker para o círculo ciano
hsv1_c = (np.array([90,50,50], dtype=np.uint8))
hsv2_c = (np.array([140,255,255], dtype=np.uint8))

In [16]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time

# Parameters to use when opening the webcam.
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

lower = 0
upper = 1

# Returns an image containing the borders of the image
# sigma is how far from the median we are setting the thresholds
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged



while(True):
    # Capture frame-by-frame
    print("New frame")
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    circulos_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # A gaussian blur to get rid of the noise in the image
    blur = cv2.GaussianBlur(gray,(5,5),0)
    #blur = gray
    # Detect the edges present in the image
    bordas = auto_canny(blur)


    circles = []


    # Obtains a version of the edges image where we can draw in color
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)

    # HoughCircles - detects circles using the Hough Method. For an explanation of
    # param1 and param2 please see an explanation here http://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
    circles = None
    circles=cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=5,maxRadius=60)

    if circles is not None:
        circles = np.uint16(np.around(circles))

        for i in circles[0,:]:
            print(i)
            # draw the outer circle
            # cv2.circle(img, center, radius, color[, thickness[, lineType[, shift]]])
            cv2.circle(bordas_color,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(bordas_color,(i[0],i[1]),2,(0,0,255),3)

    maskm = cv2.inRange(circulos_hsv, hsv1_m, hsv2_m)
    maskc = cv2.inRange(circulos_hsv, hsv1_c, hsv2_c)
            
    selecao_m = cv2.bitwise_and(frame, frame, mask= maskm)
    selecao_c = cv2.bitwise_and(frame, frame, mask= maskc)
    
    selecao = cv2.bitwise_or(selecao_m,selecao_c)
        
    # Display the resulting frame
    #cv2.imshow('Detector de circulos',bordas_color)
    cv2.imshow('Detector de circulos',selecao)
    print("No circles were found")
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

New frame
[245 145  59]
[115 155  59]
[425 225  59]
[449 139  59]
[ 43 159  57]
[175 155  46]
[ 87 105  46]
No circles were found
New frame
[249 147  59]
[115 155  59]
[ 67 159  57]
[349 127  48]
[463 129  59]
[ 95 101  41]
[171 169  41]
[ 43 231  41]
[419 259  50]
[419 165  37]
No circles were found
New frame
[115 155  59]
[247 147  59]
[ 67 161  59]
[443 143  57]
[ 87 105  46]
[171 153  48]
[401 149  48]
[ 41 237  37]
No circles were found
New frame
[115 155  59]
[245 147  59]
[443 143  57]
[425 227  54]
[ 67 159  43]
[351 145  43]
[105 103  46]
[169 169  41]
[429 273  50]
No circles were found
New frame
[115 155  59]
[249 147  59]
[ 61 159  57]
[367 143  59]
[ 89 105  46]
[413 255  50]
[167 165  37]
[437 145  43]
No circles were found
New frame
[151 143  59]
[243 147  59]
[349 127  48]
[ 59 161  54]
[111 159  54]
[105 105  46]
[437 141  48]
[ 47 229  43]
No circles were found
New frame
[247 147  59]
[115 155  59]
[ 61 159  57]
[349 127  48]
[ 91 105  46]
[175 155  46]
[375 229  59]


[247 147  59]
[109 155  59]
[ 57 157  59]
[361 157  32]
[143 179  32]
No circles were found
New frame
[247 147  59]
[115 157  57]
[ 61 155  52]
[381 165  35]
[171 161  35]
[113 101  41]
No circles were found
New frame
[253 145  59]
[117 157  57]
[ 63 159  59]
[107  99  41]
No circles were found
New frame
[247 145  59]
[109 155  59]
[153 163  48]
[ 69 147  46]
[111 105  48]
[353 143  41]
No circles were found
New frame
[247 145  59]
[ 83 157  59]
[123 153  50]
[165 151  50]
[ 31 159  57]
[363 143  46]
[115 111  41]
[503 287  50]
No circles were found
New frame
[247 145  59]
[ 55 159  57]
[123 157  57]
[353 141  46]
[167 153  50]
[97 95 37]
No circles were found
New frame
[115 155  59]
[253 147  59]
[ 55 159  57]
[349 119  59]
[103 103  46]
No circles were found
New frame
[115 155  59]
[243 147  59]
[ 61 159  57]
[115 103  46]
[371 161  28]
No circles were found
New frame
[247 147  59]
[115 155  59]
[ 61 155  52]
[179 151  41]
[113 115  37]
[361 157  30]
No circles were found
New frame
[

[467 347  26]
No circles were found
New frame
[251 141  54]
[ 87 151  50]
[ 31 159  57]
[179 155  46]
[353 147  41]
No circles were found
New frame
[251 141  54]
[113 155  59]
[ 45 161  54]
[467 347  26]
[349 145  41]
[287 349  23]
[101 111  39]
No circles were found
New frame
[247 147  59]
[ 95 159  59]
[ 55 159  57]
[171 151  46]
[349 145  41]
[115 105  46]
[287 349  23]
No circles were found
New frame
[253 145  59]
[115 157  57]
[ 73 149  48]
[353 143  43]
[173 153  46]
[111 113  37]
No circles were found
New frame
[257 147  59]
[115 159  54]
[ 53 149  48]
[349 151  41]
[181 157  43]
[ 99 103  41]
No circles were found
New frame
[249 145  59]
[ 57 157  57]
[107 155  59]
[467 349  26]
[353 147  41]
[285 349  24]
[113 111  41]
No circles were found
New frame
[251 147  59]
[115 157  57]
[ 55 159  57]
[171 155  57]
[353 141  46]
[465 349  25]
[107 109  46]
[285 353  23]
No circles were found
New frame
[253 147  59]
[361 129  59]
[115 157  57]
[ 73 149  48]
[283 353  23]
No circles were 

## Parte 3

Assumindo que a folha se mantém sempre paralela ao plano de imagem da câmera, imprima a distância entre a folha e sua câmera

In [17]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time

# Parameters to use when opening the webcam.
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

lower = 0
upper = 1

# Returns an image containing the borders of the image
# sigma is how far from the median we are setting the thresholds
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged


f = 495.08181142110243
ho = 14


while(True):
    # Capture frame-by-frame
    #print("New frame")
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # A gaussian blur to get rid of the noise in the image
    blur = cv2.GaussianBlur(gray,(5,5),0)
    #blur = gray
    # Detect the edges present in the image
    bordas = auto_canny(blur)


    circles = []
    centers = []

    
    # Obtains a version of the edges image where we can draw in color
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)

    # HoughCircles - detects circles using the Hough Method. For an explanation of
    # param1 and param2 please see an explanation here http://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
    circles = None
    circles=cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=5,maxRadius=60)

    if circles is not None:
        circles = np.uint16(np.around(circles))

        for i in circles[0,:]:
            centers.append(i)
            # draw the outer circle
            # cv2.circle(img, center, radius, color[, thickness[, lineType[, shift]]])
            cv2.circle(bordas_color,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(bordas_color,(i[0],i[1]),2,(0,0,255),3)

            #cv2.line(bordas_color,(i[0],i[1]),(i[0],i[1]),(255,0,0),5)
            
            if len(centers)==2:
                
                cv2.line(bordas_color,(centers[0][0],centers[0][1]),(centers[1][0],centers[1][1]),(255,0,0),5)
                
                #Coordenadas círculo 1
                tx1 = int(centers[0][0])
                ty1 = int(centers[0][1])

                #Coordenadas círculo 2
                tx2 = int(centers[1][0])
                ty2 = int(centers[1][1])

                #Normalizando
                tx = abs(tx2-tx1)
                ty = abs(ty2-ty1)

                #Calculando a distância entre os círculos por Pitágoras
                hi_ = math.sqrt(ty**2 + tx**2)
                d = f*ho/hi_
       
    
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(bordas_color, str(d), (0,50), font, 2,(255,255,255),2,cv2.LINE_AA)
        
    

    # Display the resulting frame
    cv2.imshow('Detector de circulos',bordas_color)
    #print("No circles were found")
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

## Parte 4

Trace uma linha entre os centros do círculo magenta e do círculo ciano.

Imprima na tela o ângulo entre esta linha e a horizontal

In [18]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time

# Parameters to use when opening the webcam.
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

lower = 0
upper = 1

# Returns an image containing the borders of the image
# sigma is how far from the median we are setting the thresholds
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged


f = 495.08181142110243
ho = 14

hsv1_m = np.array([150,50,50], dtype=np.uint8)
hsv2_m = np.array([220,255,255], dtype=np.uint8)

hsv1_c = (np.array([90,50,50], dtype=np.uint8))
hsv2_c = (np.array([140,255,255], dtype=np.uint8))


while(True):
    # Capture frame-by-frame
    #print("New frame")
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    circulos_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # A gaussian blur to get rid of the noise in the image
    blur = cv2.GaussianBlur(gray,(5,5),0)
    #blur = gray
    # Detect the edges present in the image
    bordas = auto_canny(blur)


    circles = []
    centers = []

    
    # Obtains a version of the edges image where we can draw in color
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)

    # HoughCircles - detects circles using the Hough Method. For an explanation of
    # param1 and param2 please see an explanation here http://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
    circles = None
    circles=cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=5,maxRadius=60)
    
    if circles is not None:
        circles = np.uint16(np.around(circles))

        for i in circles[0,:]:
            centers.append(i)
            # draw the outer circle
            # cv2.circle(img, center, radius, color[, thickness[, lineType[, shift]]])
            cv2.circle(bordas_color,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(bordas_color,(i[0],i[1]),2,(0,0,255),3)

            #cv2.line(bordas_color,(i[0],i[1]),(i[0],i[1]),(255,0,0),5)
            
            if len(centers)==2:
                
                cv2.line(bordas_color,(centers[0][0],centers[0][1]),(centers[1][0],centers[1][1]),(255,0,0),5)
                
                #Coordenadas círculo 1
                tx1 = int(centers[0][0])
                ty1 = int(centers[0][1])

                #Coordenadas círculo 2
                tx2 = int(centers[1][0])
                ty2 = int(centers[1][1])

                #Normalizando
                tx = abs(tx2-tx1)
                ty = abs(ty2-ty1)

                #Calculando a distância entre os círculos por Pitágoras
                hi_ = math.sqrt(ty**2 + tx**2)
                d = f*ho/hi_
    
    if tx!=0:
        angulo = math.degrees(math.atan(ty/tx))
    else:
        angulo = 90

        
    maskm = cv2.inRange(circulos_hsv, hsv1_m, hsv2_m)
    maskc = cv2.inRange(circulos_hsv, hsv1_c, hsv2_c)
            
    selecao_m = cv2.bitwise_and(frame, frame, mask= maskm)
    selecao_c = cv2.bitwise_and(frame, frame, mask= maskc)
    
    selecao = cv2.bitwise_or(selecao_m,selecao_c)
    img_4 = cv2.bitwise_or(selecao,bordas_color)    
    
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img_4, str(angulo), (0,50), font, 2,(255,255,255),2,cv2.LINE_AA)

    
    # Display the resulting frame
    cv2.imshow('Detector de circulos',img_4)
    #print("No circles were found")
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

## Parte 5

Usando transformada de Hough, desenhe um círculo sobre o círculo ciano e outro sobre o círculo magenta.

**Desafio bônus**: ser capaz de eliminar circulos espúrios (aqueles que não são os da folha)

In [19]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time

# Parameters to use when opening the webcam.
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

lower = 0
upper = 1

# Returns an image containing the borders of the image
# sigma is how far from the median we are setting the thresholds
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged


f = 495.08181142110243
ho = 14

hsv1_m = np.array([150,50,50], dtype=np.uint8)
hsv2_m = np.array([220,255,255], dtype=np.uint8)

hsv1_c = (np.array([90,50,50], dtype=np.uint8))
hsv2_c = (np.array([140,255,255], dtype=np.uint8))


while(True):
    # Capture frame-by-frame
    #print("New frame")
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    circulos_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # A gaussian blur to get rid of the noise in the image
    blur = cv2.GaussianBlur(gray,(5,5),0)
    #blur = gray
    # Detect the edges present in the image
    bordas = auto_canny(blur)


    circles = []
    centers = []

    
    # Obtains a version of the edges image where we can draw in color
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)

    # HoughCircles - detects circles using the Hough Method. For an explanation of
    # param1 and param2 please see an explanation here http://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
    circles = None
    circles=cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=5,maxRadius=60)
    
    if circles is not None:
        circles = np.uint16(np.around(circles))

        for i in circles[0,:]:
            centers.append(i)
            # draw the outer circle
            # cv2.circle(img, center, radius, color[, thickness[, lineType[, shift]]])
            cv2.circle(bordas_color,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(bordas_color,(i[0],i[1]),2,(0,0,255),3)

            #cv2.line(bordas_color,(i[0],i[1]),(i[0],i[1]),(255,0,0),5)
            
            if len(centers)==2:
                
                #cv2.line(bordas_color,(centers[0][0],centers[0][1]),(centers[1][0],centers[1][1]),(255,0,0),5)
                
                #Coordenadas círculo 1
                tx1 = int(centers[0][0])
                ty1 = int(centers[0][1])

                #Coordenadas círculo 2
                tx2 = int(centers[1][0])
                ty2 = int(centers[1][1])

                #Normalizando
                tx = abs(tx2-tx1)
                ty = abs(ty2-ty1)

                #Calculando a distância entre os círculos por Pitágoras
                hi_ = math.sqrt(ty**2 + tx**2)
                d = f*ho/hi_
       

    maskm = cv2.inRange(circulos_hsv, hsv1_m, hsv2_m)
    maskc = cv2.inRange(circulos_hsv, hsv1_c, hsv2_c)
            
    selecao_m = cv2.bitwise_and(frame, frame, mask= maskm)
    selecao_c = cv2.bitwise_and(frame, frame, mask= maskc)
    
    selecao = cv2.bitwise_or(selecao_m,selecao_c)
    
    img_4 = cv2.bitwise_or(selecao,bordas_color)    
    
    
    #font = cv2.FONT_HERSHEY_SIMPLEX
    #cv2.putText(img_4, str(d), (0,50), font, 2,(255,255,255),2,cv2.LINE_AA)

    
    # Display the resulting frame
    cv2.imshow('Detector de circulos',img_4)
    #print("No circles were found")
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

## Parte 6

Usando `SIFT`, identifique o escrito *Insper* na folha

In [20]:
# Parameters to use when opening the webcam.
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

lower = 0
upper = 1

# Número mínimo de pontos correspondentes
MIN_MATCH_COUNT = 10

img_original = cv2.imread('LOGO.png',0)      # Gabarito / Imagem a procurar



while(True):
    # Capture frame-by-frame
    print("New frame")
    ret, frame = cap.read()


    
    img_cena = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Versões RGB das imagens, para plot
    original_rgb = cv2.cvtColor(img_original, cv2.COLOR_GRAY2RGB)
    cena_rgb = cv2.cvtColor(img_cena, cv2.COLOR_GRAY2RGB)

    # Imagem de saída
    out = frame.copy()


    # Cria o detector SIFT
    sift = cv2.xfeatures2d.SIFT_create()

    # Encontra os pontos únicos (keypoints) nas duas imagems
    kp1, des1 = sift.detectAndCompute(img_original ,None)
    kp2, des2 = sift.detectAndCompute(img_cena,None)

    # Configurações do algoritmo FLANN que compara keypoints e ver correspondências - não se preocupe com isso
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    # Configura o algoritmo de casamento de features que vê *como* o objeto que deve ser encontrado aparece na imagem
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Tenta fazer a melhor comparacao usando o algoritmo
    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)


    if len(good)>MIN_MATCH_COUNT:
        # Separa os bons matches na origem e no destino
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)


        # Tenta achar uma trasformacao composta de rotacao, translacao e escala que situe uma imagem na outra
        # Esta transformação é chamada de homografia 
        # Para saber mais veja 
        # https://docs.opencv.org/3.4/d9/dab/tutorial_homography.html
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()



        h,w = img_original.shape
        # Um retângulo com as dimensões da imagem original
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

        # Transforma os pontos do retângulo para onde estao na imagem destino usando a homografia encontrada
        dst = cv2.perspectiveTransform(pts,M)


        # Desenha um contorno em vermelho ao redor de onde o objeto foi encontrado
        img2b = cv2.polylines(out,[np.int32(dst)],True,(255,0,0),3, cv2.LINE_AA)

    else:
        print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
        matchesMask = None

    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                       singlePointColor = None,
                       matchesMask = matchesMask, # draw only inliers
                       flags = 2)
    

    # Display the resulting frame
    cv2.imshow('Detector de circulos',out)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

New frame
New frame
New frame
New frame
New frame
Not enough matches are found - 8/10
New frame
Not enough matches are found - 1/10
New frame
Not enough matches are found - 3/10
New frame
Not enough matches are found - 7/10
New frame
Not enough matches are found - 7/10
New frame
Not enough matches are found - 3/10
New frame
Not enough matches are found - 6/10
New frame
Not enough matches are found - 7/10
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
New frame
